In [1]:
import os
import bs4
import tqdm
import glob
from itertools import chain

In [ ]:
<p class="dateGenerated">
    <small>calibre2opds 3.5  (rev. 369): Catalog generated on Apr 4, 2020  (406 books)</small>
</p>

In [20]:
files = glob.glob("docs/*.html")

In [21]:
files = [files[0]]

In [22]:
for file in tqdm.tqdm(files):
    try:    
        with open(file,"r") as f:
            soup = bs4.BeautifulSoup(f.read(),features="lxml")
            p = soup.select(".dateGenerated > small")[0]
            p.string = f"Last updated on {p.string.split('on')[1]}"
#         with open(file,"w") as f:    
#             f.write(str(soup))
    except Exception as e:
        print(f"error formatting last updated {file}",e)

100%|██████████| 1/1 [00:00<00:00, 453.14it/s]

error formatting last updated docs/header.html 'NoneType' object has no attribute 'decompose'


In [40]:
soup

<!DOCTYPE html PUBLIC "-//W3c//DTD html 4.01//EN" "http://www.w3c.org/tr/html4/strict.dtd">
<html><body><p class="dateGenerated">
<small>Last updated on  Apr 4, 2020  (406 books)</small>
</p>
</body></html>

In [5]:
print("Deleting footers")

Deleting all other footers


In [7]:
#files = chain(glob.glob("docs/*/*.html"), glob.glob("docs/*.html"))
files = list(glob.glob("docs/*/*.html"))
files.extend(glob.glob("docs/*.html"))
for file in tqdm.tqdm(files):
    try:    
        with open(file,"r") as f:
            soup = bs4.BeautifulSoup(f.read(),features="lxml")
            soup.find(id="footer").decompose()
        with open(file,"w") as f:    
            f.write(str(soup))
    except Exception as e:
        if file not in  ["docs/search/search.html","docs/header.html"]:
            print(f"error deleting footer html {file}",e)

100%|██████████| 881/881 [00:05<00:00, 156.18it/s]


In [8]:
print("Fixing Titles")

Fixing Title


In [9]:
file_path  = [f"./docs/book_0/{i}" for i in os.listdir("docs/book_0") if i[-4:]=="html"]

In [10]:
author_t = " div.fullEntry_title > h1 > small > em > small > em > a"
title_t = " div.fullEntry_title > h1"

In [11]:
file = file_path[1]

In [12]:
for file in tqdm.tqdm(file_path):
    try:    
        with open(file,"r") as f:
            soup = bs4.BeautifulSoup(f.read(),features="lxml")
            p = soup.select(".x_container")[0]
            author = p.select(author_t)[0].text
            title = p.select(title_t)[0].text.split("\n")[1].strip()
            soup.head.title.string = f"{title}, By {author} | {soup.head.title.string}"
        with open(file,"w") as f:    
            f.write(str(soup))
    except Exception as e:
         print(f"error: html {file}",e,file)
        


  5%|▌         | 21/406 [00:00<00:03, 100.00it/s]

error: html ./docs/book_0/book_725.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_725.html
error: html ./docs/book_0/book_540.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_540.html
error: html ./docs/book_0/book_27.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_27.html
error: html ./docs/book_0/book_752.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_752.html
error: html ./docs/book_0/book_149.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_149.html
error: html ./docs/book_0/book_689.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_689.html
error: html ./docs/book_0/book_112.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_112.html
error: html ./docs/book_0/book_624.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_624.html
error: html ./docs/book_0/book_661.html 'NoneType' object has no attribute 'decomp

 11%|█         | 44/406 [00:00<00:03, 103.84it/s]

error: html ./docs/book_0/book_400.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_400.html
error: html ./docs/book_0/book_601.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_601.html
error: html ./docs/book_0/book_559.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_559.html
error: html ./docs/book_0/book_434.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_434.html
error: html ./docs/book_0/book_469.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_469.html
error: html ./docs/book_0/book_10.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_10.html
error: html ./docs/book_0/book_546.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_546.html
error: html ./docs/book_0/book_596.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_596.html
error: html ./docs/book_0/book_38.html 'NoneType' object has no attribute 'decompo

 17%|█▋        | 67/406 [00:00<00:03, 108.34it/s]

...

error: html ./docs/book_0/book_604.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_604.html
error: html ./docs/book_0/book_36.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_36.html
error: html ./docs/book_0/book_524.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_524.html
error: html ./docs/book_0/book_61.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_61.html
error: html ./docs/book_0/book_541.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_541.html
error: html ./docs/book_0/book_656.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_656.html
error: html ./docs/book_0/book_525.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_525.html
error: html ./docs/book_0/book_695.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_695.html
error: html ./docs/book_0/book_464.html 'NoneType' object has no attribute 'decompos

 94%|█████████▎| 380/406 [00:03<00:00, 85.31it/s] 

error: html ./docs/book_0/book_471.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_471.html
error: html ./docs/book_0/book_730.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_730.html
error: html ./docs/book_0/book_599.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_599.html
error: html ./docs/book_0/book_460.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_460.html
error: html ./docs/book_0/book_499.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_499.html
error: html ./docs/book_0/book_649.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_649.html
error: html ./docs/book_0/book_398.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_398.html
error: html ./docs/book_0/book_724.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_724.html
error: html ./docs/book_0/book_28.html 'NoneType' object has no attribute 'decom

100%|██████████| 406/406 [00:04<00:00, 98.23it/s]

error: html ./docs/book_0/book_307.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_307.html
error: html ./docs/book_0/book_98.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_98.html
error: html ./docs/book_0/book_15.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_15.html
error: html ./docs/book_0/book_468.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_468.html
error: html ./docs/book_0/book_751.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_751.html
error: html ./docs/book_0/book_71.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_71.html
error: html ./docs/book_0/book_667.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_667.html
error: html ./docs/book_0/book_428.html 'NoneType' object has no attribute 'decompose' ./docs/book_0/book_428.html
error: html ./docs/book_0/book_155.html 'NoneType' object has no attribute 'decompose'